In [1]:
import requests 
from openai import OpenAI
from qdrant_client import QdrantClient, models

/home/endie/Projects/LLM-Projects/LLM-Zoomcamp/LLM-workSpace/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
openai_client = OpenAI()

## Objective
You're setting up a Qdrant vector database to store embeddings (512-dimensional) from a model (jinaai/jina-embeddings-v2-small-en) for a collection named "zoomcamp-faq". This prepares the collection to perform similarity search using cosine distance.

- Imports the `QdrantClient` to interact with the Qdrant database.
- `models` gives access to configuration classes like `VectorParams`, `Distance`, etc.

---

####
```python
qd_client = QdrantClient("http://localhost:6333")
```


In [9]:
qd_client = QdrantClient("http://localhost:6333")

### EMBEDDING_DIMENSIONALITY = 512
- Defines the size of the embedding vectors you'll store.
- This must match the output size of your embedding model — in this case, `jinaai/jina-embeddings-v2-small-en`.

```python
model_handle = "jinaai/jina-embeddings-v2-small-en"
```
---
### collection_name = "zoomcamp-faq"
- Name of the **collection** (like a table or index) in Qdrant that will hold the embeddings.

 ```python
qd_client.delete_collection(collection_name=collection_name)
```

+ Deletes the collection if it exists — useful during development or reinitialization.

+ Prevents duplicate schema errors and clears previous data.

In [10]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"
collection_name = "zoomcamp-faq"

In [11]:
qd_client.delete_collection(collection_name=collection_name)

True

### Create the collection
- Creates a new collection ready to store 512-dimension embeddings
- Configures it to use **cosine similarity** for search

You can now insert embeddings into this collection and perform nearest-neighbor vector searches.


In [12]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

### create_payload_index

- You're telling Qdrant to **index** the `"course"` field in the documents' payloads so you can:
- **Efficiently filter** your vector search results (e.g., only search within `"course": "LLM-Zoomcamp"`)
- Use **non-vector metadata** in hybrid queries (e.g., `"must match this topic"`)

---

### 🔍 What Each Parameter Means:

| Argument | Description |
|----------|-------------|
| `collection_name=collection_name` | You're applying this index to your `"zoomcamp-faq"` collection. |
| `field_name="course"` | You’re indexing the `"course"` field from the **payload** attached to each vector. |
| `field_schema="keyword"` | Qdrant should treat this as a **discrete, filterable string**, like a tag or category. |

> 🔑 A “payload” in Qdrant is like metadata — additional info stored with the vector (e.g., question text, topic, source, etc.)

---

### ✅ Result

After this line runs:
- You can do fast filtered searches like:
  ```python
  filter=models.Filter(
      must=[models.FieldCondition(
          key="course",
          match=models.MatchValue(value="LLM-Zoomcamp")
      )]
  )


In [13]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [14]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [15]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

### Study Data Visually

Let’s explore the uploaded data in the Qdrant Web UI at [http://localhost:6333/dashboard](http://localhost:6333/dashboard) to study semantic similarity visually.

For example, using the `Visualize` tab in the `zoomcamp-rag` collection, we can view all answers to the course questions (948 points) and see how they group together by meaning, additionally coloured by the course type.  

To do that, run the following command:

```json
{
  "limit": 948,
  "color_by": {
    "payload": "course"
  }
}
```

This 2D representation is the result of dimensionality reduction applied to `jina-embeddings`.

In [16]:
question = 'I just discovered the course. Can I still join it?'

In [17]:
def vector_search(question):
    print('vector_search is used')
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( 
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [18]:
vector_search(question)

vector_search is used


[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [20]:
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [21]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [22]:
rag(question)

vector_search is used


"Yes, you can still join the course even after the start date. You are eligible to submit homeworks, but keep in mind that there will be deadlines for final projects, so it's advisable not to leave everything for the last minute."

In [23]:
rag('how do I run kafka?')

vector_search is used


"To run Kafka, you need to follow these steps:\n\n1. **Ensure Kafka Broker is Running**: First, check if your Kafka broker docker container is working by running the command `docker ps`. If it's not running, navigate to the docker compose YAML file folder and execute `docker compose up -d` to start all the instances.\n\n2. **Running your Kafka Applications**: In your project directory, you can run your Kafka producer or consumer using the following command:\n   ```\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Replace `<jar_name>` with the actual name of your JAR file.\n\n3. **Check Configuration**: Make sure that:\n   - The `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` is set to the correct server URL in your Java scripts.\n   - Update the cluster key and secrets in `src/main/java/org/example/Secrets.java` accordingly.\n\nFollowing these steps should enable you to run Kafka successfully."